# 🥑 ruDALLE: Generate images from texts.

**Link to this notebook - UnenBubb-ruDALLE-generation**-v2.0-dualRAM 🐏🐏
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UnendingBubbles/generative-notebooks/blob/main/UnenBubb-ruDALL-E-generation.ipynb)

♥ Thank you goes to all the creators of ruDALL-E - [GitHub](https://github.com/sberbank-ai/ru-dalle) 

**Official notebooks - ruDALLE-example-generation.ipynb**
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wGE-046et27oHvNlBNPH07qrEQNE04PQ?usp=sharing) (for latest updates)

**ruDALL-E Malevich (XL) with 3.5GB vRAM!**
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1AoolDYePUpPkRCKIu0cP9zV7lX5QGD3Z?usp=sharing) (for latest updates)


helpful subreddits: [r/MachineLearning](https://www.reddit.com/r/MachineLearning/) - [r/MediaSynthesis](https://www.reddit.com/r/MediaSynthesis/) - [r/bigsleep](https://www.reddit.com/r/bigsleep/)

[UnendingBubbles](https://github.com/UnendingBubbles) - advanced image grid, live previews, QOL improvments, dual RAM workflows - last updated: 211111 v2.1

Instructions at the bottom.

In [ ]:
#@markdown #**🧠 Check Resources**
import multiprocessing
import torch
from psutil import virtual_memory
!nvidia-smi -L
gpu_ram = round(torch.cuda.get_device_properties(0).total_memory / 2**30, 2)
print(f'device: {torch.device("cuda:0" if torch.cuda.is_available() else "cpu").type} ~ vRAM: {gpu_ram} GB ~ CPU: {multiprocessing.cpu_count()} cores ~ CPU-RAM: {round(virtual_memory().total / 1024**3, 1)} GB ~ PyTorch-version: {torch.__version__} ~ CUDA version: {torch.version.cuda} ~ cuDNN version: {torch.backends.cudnn.version()}')
# !nvidia-smi

In [ ]:
#@markdown #**📘 Install ruDALL-E and Setup** (~3min)
#@markdown Turn it up to 11 (unrestricted), if you have `High-RAM` Google Colab access.

max_vram = 6  #@param {type:"slider", min:3.5, max:11.0, step:0.5}
ALLOWED_MEMORY = max_vram
dwt_mode = "True = 512x512 (artifacts)"  #@param ["False = 256x256 high-quality", "True = 512x512>256x256", "True = 512x512 (artifacts)"]
#@markdown Turn dwt True for lower RAM to avoid tensor error.

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 170})'''))  #limit output height
print('install is in progress...')

# !pip install rudalle==0.0.1rc6 > /dev/null
# !pip install rudalle==0.0.1rc7 > /dev/null
# !pip install rudalle==0.0.1rc8 > /dev/null
!pip install rudalle==0.0.1rc10 > /dev/null
# !pip3 install git+https://github.com/sberbank-ai/ru-dalle.git@master

import transformers
import more_itertools
from tqdm.auto import tqdm
from rudalle.pipelines import show, cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae, get_ruclip
from rudalle.utils import seed_everything, torch_tensors_to_pil_list
import multiprocessing
import torch
from psutil import virtual_memory

device = 'cuda'
dalle = get_rudalle_model('Malevich', pretrained=True, fp16=True, device=device)
tokenizer = get_tokenizer()
total_memory = torch.cuda.get_device_properties(0).total_memory / 2**30

if ALLOWED_MEMORY < 10.5: # set low-ram workflow
    DALLE_BS = int(ALLOWED_MEMORY-2.5)
    if torch.__version__ >= '1.8.0':
        low_ram_workflow = True
        k = ALLOWED_MEMORY/ total_memory
        torch.cuda.set_per_process_memory_fraction(k, 0)
        print('Allowed GPU RAM:', round(ALLOWED_MEMORY, 2), 'Gb')
        print('GPU part', round(k, 4))

else: # set high-ram workflow
    low_ram_workflow = False
    from rudalle.pipelines import generate_images, super_resolution
    from rudalle import get_realesrgan
    realesrgan = get_realesrgan('x2', device=device) # x2/x4/x8
    DALLE_BS = 8

# low_ram_workflow = True
if dwt_mode == "False = 256x256 high-quality":
  # vae = get_vae().to(device)  #for default 256x256 in >rc7
  if low_ram_workflow == False:
    vae = get_vae(dwt=False).to(device)  #for default 256x256 in rc7+
  else:
    vae = get_vae(dwt=False)
if dwt_mode == "True = 512x512>256x256" or dwt_mode == "True = 512x512 (artifacts)":
  if low_ram_workflow == False:
    vae = get_vae(dwt=True).to(device)   #for 512x512
  else:
    vae = get_vae(dwt=True)
ruclip, ruclip_processor = get_ruclip('ruclip-vit-base-patch32-v5')
if low_ram_workflow == False:
  ruclip = ruclip.to(device)


!pip install -U deep_translator
import time
import numpy as np
from deep_translator import GoogleTranslator, MyMemoryTranslator
# langs_dict = GoogleTranslator.get_supported_languages(as_dict=True)
# print(langs_dict)
!wget -nc https://www.1001fonts.com/download/font/open-sans.light.ttf -P /content/
from PIL import Image

def dimr(text, r=165, g=165, b=165):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)
def dark(text, r=115, g=115, b=115):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

if low_ram_workflow == True:
  def generate_codebooks(text, tokenizer, dalle, top_k, top_p, images_num, image_prompts=None, temperature=1.0, bs=8,
                      seed=None, use_cache=True):
      vocab_size = dalle.get_param('vocab_size')
      text_seq_length = dalle.get_param('text_seq_length')
      image_seq_length = dalle.get_param('image_seq_length')
      total_seq_length = dalle.get_param('total_seq_length')
      device = dalle.get_param('device')
      text = text.lower().strip()
      input_ids = tokenizer.encode_text(text, text_seq_length=text_seq_length)
      codebooks = []
      for chunk in more_itertools.chunked(range(images_num), bs):
          chunk_bs = len(chunk)
          with torch.no_grad():
              attention_mask = torch.tril(torch.ones((chunk_bs, 1, total_seq_length, total_seq_length), device=device))
              out = input_ids.unsqueeze(0).repeat(chunk_bs, 1).to(device)
              has_cache = False
              if image_prompts is not None:
                  prompts_idx, prompts = image_prompts.image_prompts_idx, image_prompts.image_prompts
                  prompts = prompts.repeat(chunk_bs, 1)
              for idx in tqdm(range(out.shape[1], total_seq_length)):
                  idx -= text_seq_length
                  if image_prompts is not None and idx in prompts_idx:
                      out = torch.cat((out, prompts[:, idx].unsqueeze(1)), dim=-1)
                  else:
                      logits, has_cache = dalle(out, attention_mask,
                                                has_cache=has_cache, use_cache=use_cache, return_loss=False)
                      logits = logits[:, -1, vocab_size:]
                      logits /= temperature
                      filtered_logits = transformers.top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)
                      probs = torch.nn.functional.softmax(filtered_logits, dim=-1)
                      sample = torch.multinomial(probs, 1)
                      out = torch.cat((out, sample), dim=-1)
              codebooks.append(out[:, -image_seq_length:].cpu())
      return codebooks

# if you want to save to google drive, run that codeblock (near bottom) before continuing

In [ ]:
# text = 'a sturdy red chair'
# text = 'thinking woman statue logo.'
# text = "'Calm Sailing' a popular classic oil painting of boats on the ocaean."
# text = "'Land Ahoy' a popular classic oil painting of a boat on the ocaean at sunset."
# text = 'Image of the Earth.'
# text = 'World Map'
# text = "Jungle Illustration"
# text = "'Coral Reef with Fish' - digital painting"
# text = "'Coral Reef with Irridecent Turtle' - digital painting"
# text = 'Пингвины радуются - неизвестная картина Казимира Малевича'
# text = "chrome bar trolley"  #temp1.05

text = "'Coral Reef with Fish' - digital painting"

original = text
tService = GoogleTranslator #GoogleTranslator, MyMemoryTranslator
translated = tService(source='en', target='ru').translate(text)
rev_translated = tService(source='ru', target='en').translate(translated)
print(dimr(f'original: {original}\ntranslted: {translated}\nrev-tran: {rev_translated}'))

text = translated  

In [ ]:
#@markdown #**🏃‍♀️ Run ruDALL-E**
# text = ""

ks = "4000,1000" #@param {type:"string"}
ps = "0.99, 0.9, 0.7" #@param {type:"string"}


temperature =  1.00#@param 1.0 {type:"number"}

n_bpr =  1#@param {type:"integer"}
seed =  -1#@param {type:"integer"}
display_progress = True #@param {type:"boolean"}

ks = [int(x) for x in ks.split(',')]
ps = [float(x) for x in ps.split(',')]

##/////////////////////////////////////
# custom pk matrix

# uncomment any to override above settings

# ks = [16384,8192,4096,2048,1024,512,256,128,56,24,12]
# ps = [1.4, 1.2, 1.0, 0.9995, 0.995, 0.99, 0.98, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]

# ks = [128000,32000,16000,8000,4000,2000,1000,600,300,160,80,40,20,10,5]
# ps = [99.0, 0.9999, 0.9995, 0.999, 0.99, 0.96, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

ks = [128000,32000,16000,8000,4000,2000,1000,600,300,160,80,40,20,10]
ps = [99.0, 0.9999, 0.9995, 0.999, 0.99, 0.96, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

# ks = [128000,32000,8000,4000,1000,600,300,80,10]
# ps = [99.0, 0.9999, 0.999, 0.99, 0.96, 0.9, 0.7, 0.3]

# ks = [128000,4000,1000,600,80]
# ps = [0.9999, 0.999, 0.98, 0.9, 0.65, 0.35]

# ks = [8000,3000,600]
# ps = [0.9999, 0.9, 0.8]

##/////////////////////////////////////
# output UI and param setup

b_resolves = []
for y in range(0, len(ks)):  
  for x in range(0, len(ps)):  
    t = [(ks[y], ps[x], n_bpr)]
    b_resolves.extend(list(t))

images_num =  n_bpr
total_immys = (len(ps)) * (len(ks)) * images_num
timestart = time.strftime('%Y%m%d-%H%M%S')
if seed == -1:
  seed = np.random.randint(0, 2**31)
seed_everything(seed)
pil_images = []
pil_images2 = []
pil_images_debug512 = []
scores = []
codebooks = []
wait_time = 1
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 600})'''))  #limit output height
print(dark(b_resolves))
print(dark(f'seed: {seed}\ntotal images: {total_immys}'))
import time
t1 = time.perf_counter()  #start timer
print(dimr(f'original: {original}\ntranslted: {translated}\nrev-tran: {rev_translated}\ntext-used: {text}'))
numon = 1*images_num

##/////////////////////////////////////
# batching loop

for top_k, top_p, images_num in  b_resolves:

    if low_ram_workflow == True: #dwt = True
      scodebook = generate_codebooks(text, tokenizer, dalle, top_k=top_k, images_num=images_num, top_p=top_p, bs=DALLE_BS)
      for _codebooks in tqdm(torch.cat(scodebook).cpu()):
          with torch.no_grad():
              images = vae.decode(_codebooks.unsqueeze(0))
              spil_image = torch_tensors_to_pil_list(images)
              # pil_images_debug512 += spil_image #save 512s to pil_images_debug512
              pil_images_debug512.append(spil_image) #save 512s to pil_images_debug512
      codebooks += scodebook

      if dwt_mode == "True = 512x512>256x256":
        for num, im in enumerate(spil_image):
            im.thumbnail((256,256), resample=Image.LANCZOS, reducing_gap=3)
            pil_images.append(im) #save 256s in pil_images
      if dwt_mode == "True = 512x512 (artifacts)":
        for num, im in enumerate(spil_image):
            pil_images.append(im) #save 512s in pil_images

      if display_progress == True:
        cheapeta = format((total_immys-numon)*(72/60), ".3g")  #put your time to complete one img here (seconds/60)
        print(dark(f'{top_k} {top_p}   ~  {numon}/{total_immys}  ~   eta: {cheapeta} min'))
        show(pil_images, len(ps)*n_bpr, size=6)  #change size of output images here (network intensive if too big)
        numon += 1 * images_num

    else: #dwt = False
      _pil_images, _scores = generate_images(text, tokenizer, dalle, vae, top_k=top_k, top_p=top_p, images_num=images_num, temperature=temperature)
      pil_images += _pil_images
      scores += _scores
      if display_progress == True:
        cheapeta = format((total_immys-numon)*(50/60), ".3g")  #put your time to complete one img here (seconds/60)
        print(dark(f'{top_k} {top_p}   ~  {numon}/{total_immys}  ~   eta: {cheapeta} min'))
        show(pil_images, len(ps)*n_bpr, size=6)  #change size of output images here (network intensive if too big)
        numon += 1 * images_num


t2 = time.perf_counter()  #end timer
wait_time = t2-t1
# print('time taken to run: ',format(wait_time, ".5g"))
print(dark(f'\nimages made: {len(pil_images)}'))
print(dark(f'time taken: {time.strftime("%M:%S",time.gmtime(int(wait_time)))}'))
print(dark(f'time per image: {time.strftime("%M:%S",time.gmtime(int(wait_time)/len(pil_images)))}'))

In [ ]:
#@title 🌟 Upscale `pil_images` with ruRealEsrgan to `sr_images`
# upscale not yet avaliable (simultaniously) for low-ram
generate_sr = False #@param {type:"boolean"}
rurealesrgan_multiplier = "x2" #@param ["x2","x4","x8"]
if generate_sr == True: 
  realesrgan = get_realesrgan(rurealesrgan_multiplier, device=device)
  sr_images = super_resolution(pil_images, realesrgan)

In [ ]:
#@markdown #**🔳 Create a Grid of Images**

import time
import PIL, os, glob
from IPython.display import Image
from PIL import Image, ImageEnhance
from PIL import ImageDraw
from math import ceil, floor, trunc
from google.colab import files

generate_grid = True #@param {type:"boolean"}
draw_text = True #@param {type:"boolean"}
draw_pk_text = True #@param {type:"boolean"}
filetype_out = "png" #@param ["jpg", "png"]
img_source = "pil_images" #@param ["pil_images", "sr_images", "top_images", "pil_images_debug512"]
# filetype_in = "png" #@param ["jpg", "png"]
destination = "/content/drive/MyDrive/happyml/ruDALL-E-rc10" #@param {type:"string"}
# text is experimental and in need of more robust code
author_text = "github.com/UnendingBubbles" #@param {type:"string"}
max_imgs =  199 #@param {type:"number"}

# destination = "/content/grided"
destination_presharp = "/content/finals-sharp"

#@markdown Enable `pixel_perfect` to disable resizing of images. Must be disabled to use `frame_width`, a custom final px width. Even with `pixel_perfect` enabled, images that are different aspect ratio or larger size from the first, will be resized to fit. Must be enabled to `draw_text`.
pixel_perfect = True #@param {type:"boolean"}
#@markdown Disable `draw_pk_text` to use `images_per_row`.
images_per_row = 6 #@param {type:"integer"}
frame_width =  1200 #@param {type:"number"}
padding =   18#@param {type:"number"}
border =  42#@param {type:"number"}
bgRed = 30 #@param {type:"number"} 
bgGreen = 30 #@param {type:"number"} 
bgBlue =  30 #@param {type:"number"} 

download_when_complete = True  #@param {type:"boolean"}

#@markdown ___
#@markdown **Sharpening:** might only be useful if images are generated from non-sharpend datasets, very downscaled, or viewed on an imperfect medium. Sharpening is always a destructive process, but so is looking at a computer monitor.

#@markdown (Values above 1.00 sharpen, below 1.00 to blur.)

#@markdown **Pre-sharpening** to be applied to original input images, durring processing, before images are put into a grid.

apply_pre_sharp = False #@param {type:"boolean"}
#@markdown  (Values of 1.10-1.50 when downscaling. Less if `pixel_perfect` enabled)
pre_sharpening =  1.6#@param {type:"number"}  
#@markdown  Save all those pre-sharpened originals in `finals-sharp` folder?
save_pre_sharps = False #@param {type:"boolean"}

#@markdown If you can notice sharpening without zooming to 1:1 pixels, it's probably too much sharpening. 🤷‍♀️ Windows tip: press Ctrl+1 in windows-photos-app to view at 100% scale, after opening.

##/////////////////////////////////////
# setup params

timegrid = time.strftime('%H%M%S')
reso_crp = len(ps)+1 #max length of info-text b_resolve list (any number)
reso_cropped = (str(b_resolves[:reso_crp]) + '...') if len(b_resolves) > reso_crp else b_resolves

if draw_text == True:
  pad_bottom =  30
else:
  pad_bottom =  0

if draw_pk_text == True:
  images_per_row = len(ps)*n_bpr  #override

text_safe = (f'{rev_translated[:15]}')

# img_source = globals()[img_source] #convert string to loaded images
img_source = eval(img_source) #convert string to loaded images (less safe, but handles index)
# pil_tup = tuple(img_source) #make list of PIL imgs into enumeration-compliant tupple
# images = glob.glob("/content/finll/*.*")
# images = pil_images
# images = sr_images
# images = pil_tup
images = img_source
images = images[:max_imgs]

os.makedirs(f'{destination}', exist_ok=True)
full_destination = f'{destination}/{timestart}-{text_safe}-{timegrid}-{str(len(images))}up.{filetype_out}'

##/////////////////////////////////////
# image placement

def gridcreator(destination, frame_width):

  # images = glob.glob("/content/finals/*.*")
  # images = pil_tup
  # images = pil_images
  # images = sr_images
  images = img_source
  images = images[:max_imgs]
  # images.sort(key=os.path.getctime)      #sort files by date

  img_width, img_height = images[0].size
  sf = (frame_width-(images_per_row-1)*padding)/(images_per_row*img_width)     #scaling factor
  scaled_img_width = ceil(img_width*sf)                  
  scaled_img_height = ceil(img_height*sf) + padding
  number_of_rows = ceil(len(images)/images_per_row)

  if pixel_perfect == True:
    scaled_img_width = img_width
    scaled_img_height = img_height + padding
    frame_width = images_per_row * (img_width) + ((images_per_row-1) * padding)

  frame_height = ceil(scaled_img_height*number_of_rows)
  new_im = Image.new('RGB', (frame_width+border*2, frame_height+border*2-padding+pad_bottom), (bgRed, bgGreen, bgBlue)) 

  i,j=0,0
  for num, im in enumerate(images):
      if num%images_per_row==0:
          i=0
      # im = Image.open(im) if using saved images, load them as PIL
      if apply_pre_sharp == 1:
        enhancer = ImageEnhance.Sharpness(im)
        im = enhancer.enhance(pre_sharpening)
        if save_pre_sharps == 1:
          os.makedirs(f'{destination_presharp}', exist_ok=True)
          if filetype_out == "jpg":
            new_im.save(f'{destination_presharp}/{text_safe}-{i}', "JPEG", quality=94, optimize=True, progressive=True)
          if filetype_out == "png":
            new_im.save(f'{destination_presharp}/{text_safe}-{i}', "PNG", quality=87, optimize=True, progressive=True)
      im.thumbnail((scaled_img_width,scaled_img_height), resample=Image.LANCZOS, reducing_gap=3)
      y_cord = (j//images_per_row)*scaled_img_height
      new_im.paste(im, (i+border,y_cord+border))
      #print(i, y_cord)
      i=(i+scaled_img_width)+padding
      j+=1

##/////////////////////////////////////
# draw text

  if apply_pre_sharp == True:
    sharp_info =  f' - pre-sharp={apply_pre_sharp}:{pre_sharpening}'
  else:
    sharp_info = ""

  if draw_text == True:
    mygrey = 75
    mygrey2 = 100
    mycolr = (mygrey, mygrey, mygrey)
    mycolr2 = (mygrey2, mygrey2, mygrey2)
    from PIL import ImageFont
    # !wget -nc https://www.1001fonts.com/download/font/open-sans.light.ttf -P /content/
    font = ImageFont.truetype(r'/content/open-sans.light.ttf', 11)
    draw = ImageDraw.Draw(new_im)
    # draw.text((border+1, frame_height+border+(number_of_rows-1)*padding-40),f'{original} - {translated}',mycolr,font=font)
    # draw.text((border+1, frame_height+border+(number_of_rows-1)*padding-20),f'utc{timestart} - s{seed} - {format(wait_time/60, ".3g")}min - temp{temperature} - {reso_cropped}{sharp_info} - ruDALL-E-rc10 - {author_text}',mycolr,font=font)
    draw.text((border+1, frame_height+border-5),f'{original} - {translated}',mycolr,font=font)
    draw.text((border+1, frame_height+border+13),f'utc{timestart} - s{seed} - {format(wait_time/60, ".3g")}min - temp{temperature} - {reso_cropped}{sharp_info} - ruDALL-E-rc10 - {author_text}',mycolr,font=font)

    if draw_pk_text == True:
      draw.text((border+1,border-30),f'[ k - p ] grid for ruDALL-E',mycolr2,font=font) 
      for y in range(0, number_of_rows): 
        for x in range(0, len(ps)): 
          draw.text(((scaled_img_width+padding)*x*n_bpr+border,(scaled_img_height)*y-15+border),f'{ks[y]} - {ps[x]}',mycolr2,font=font)

##/////////////////////////////////////
# saving

  if filetype_out == "jpg":
    new_im.save(full_destination, "JPEG", quality=94, optimize=True, progressive=True)
  if filetype_out == "png":
    new_im.save(full_destination, "PNG", quality=87, optimize=True, progressive=True)
  if download_when_complete == True:
    print('download requet sent...')
    files.download(full_destination)

  print(f"{full_destination} - {os.path.getsize(full_destination)/1048576:.3f} MB")

##/////////////////////////////////////
# generate

try:
  if generate_grid == True:
    #print('generating grid...')
    gridcreator(destination, frame_width)  #it gets upset if you dont pass these variables for some reason
    #Image(full_destination)
  else:
    print('check "generate_grid" to run this feature.')
except KeyboardInterrupt:  #ability to stop without unresponsive error
  pass

print(text)
full_destination = f'{destination}/{text_safe}-{str(len(images))}up.{filetype_out}'
print("\n")
# Warning: Colab session may softcrash if trying to display very high-res files. It sends uncompressed PNG to browser no matter the input. Uncomment below for preview (if you dare).
# from IPython.display import Image 
# Image(full_destination)

In [ ]:
#@title 💾 Export `pil_images` as PNG images
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))  #limit output height
from google.colab import files
import os
save_out = True #@param {type:"boolean"}
zip_out = True #@param {type:"boolean"}
out_destination = "/content/outs" #@param {type:"string"}
zip_destination = "/content/zips" #@param {type:"string"}
quality = 87 #@param {type:"integer"}
download_zip = True  #@param {type:"boolean"}
if save_out == True:
  timesave = time.strftime('%H%M%S')
  text_safe = (f'{rev_translated[:15]}')
  text_safe = text_safe.replace(" ", "-") #remove space
  text_safe = text_safe.replace("'", "-") #remove special
  os.makedirs(f'{out_destination}', exist_ok=True)
  for i in range(0, len(pil_images)):
    im = pil_images[i]
    im.save(f'{out_destination}/{timestart}-{text_safe}-{timesave}-{i}.png', "PNG", quality=quality, optimize=True, progressive=True)
if zip_out == True:
    zip_place = f'{zip_destination}-{timestart}-{text_safe}-{timesave}.zip'
    !zip -r $zip_place $out_destination
    from google.colab import files
    files.download(zip_place)

In [ ]:
#@title 💾 Zip a Folder
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))  #limit output height
from google.colab import files
import os
zip_folder = False #@param {type:"boolean"}
folder_to_zip = "/content/grided" #@param {type:"string"}
download_zip = False  #@param {type:"boolean"}
if zip_folder == True:
    zip_place = f'{zip_destination}-{timestart}-{timesave}.zip'
    !zip -r $folder_to_zip $out_destination
    from google.colab import files
    files.download(zip_place)

In [ ]:
#@title 💾 Mount Google Drive (optional)
#@markdown To connect Google Drive, set `root_path` to the relative drive folder path you want outputs to be saved to if you already made a directory, then execute this cell.
mount_gdrive = False #@param {type:"boolean"}
if mount_gdrive == True: 
  import os
  root_path = 'happyml/ruDALL-E-rc10' #@param {type: "string"}
  gdrive_root_path = ""
  if len(root_path) > 0:
      gdrive_root_path = '/content/drive/MyDrive/' + root_path
  from google.colab import drive
  drive.mount('/content/drive')

  if len(gdrive_root_path) > 0:
      # os.chdir(gdrive_root_path) # Changes directory to absolute root path
      print(f'gdrive_root_path = {gdrive_root_path}')

#@markdown When asked, in output below, paste your code and hit Enter.

#@markdown Paste this directory into the "destination" of the Grid Generator.

In [ ]:
#@title 🍒Cherry-pick `pil_images` with ruCLIP to `top_images` 
pick_cherries = False #@param {type:"boolean"}
n_cherries = 6 #@param {type:"integer"}
if pick_cherries == True: 
  top_images, clip_scores = cherry_pick_by_clip(pil_images, text, ruclip, ruclip_processor, device=device, count=n_cherries)
  show(top_images, 4)  #4 columns wide

In [ ]:
#@title 🌟 Upscale `top_images` with ruRealEsrgan to `sr_images`
generate_sr = False #@param {type:"boolean"}
rurealesrgan_multiplier = "x2" #@param ["x2","x4","x8"]
if generate_sr == True: 
  realesrgan = get_realesrgan(rurealesrgan_multiplier, device=device)
  sr_images = super_resolution(top_images, realesrgan)

In [ ]:
#@title Display SR images
# show(sr_images, 3, save_dir='/content/sr')

In [ ]:
#@title Display images in sorted order
# show([pil_image for pil_image, score in sorted(zip(pil_images, scores), key=lambda x: -x[1])] , 6)

---
Create an editable copy for yourself with `File > Save a Copy in Drive`.  
To Run this Notebook, select `Runtime` in the top toolbar menu, then `Run All (Ctrl+F9)`.   
To Run/Stop individual code blocks: Click the `▶` or `⏹` button in the upper-left of each code block. It will appear when hovering. Some codeblocks require a `☑` checked to run (Run-All compliant).

---
If you lose connection, or Colab is acting weird, you can save your settings with `File > Download > .ipynb`.  
Then, refresh with `F5`. If that does not work, try `Runtime > Restart Runtime` or `Factory Restart Runtime`.  

---
You can doublecheck Colab settings with `Runtime > Change Runstime Type`:  
* `Hardware Accelerator` must be `GPU`.
* `Runtime Shape` should be `High-RAM` if avaliable.
* `☑ Omit code cell output when saving` should be checked so that your `.ipynb` files do not grow massive in size.

---
This notebook can output `error` or `warning`, but if it continues, it's okay.  
If CUDA runs out of memory, try a factory restart with all default settings.